In [0]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.utils import np_utils

In [0]:
num_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(np.append(x_train.shape, 1))
x_test = x_test.reshape(np.append(x_test.shape, 1))

encode_y_train = np_utils.to_categorical(y_train)
encode_y_test = np_utils.to_categorical(y_test)

train_seed = x_train[:1000]
train_unlabelled = x_train[1000:]
encode_y_train_seed = encode_y_train[:1000]
encode_y_train_unlabelled = encode_y_train[1000:]

In [0]:
class convnet():
  def __init__(self):    
    self.model = Sequential()
    self.model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    self.model.add(Conv2D(64, (3, 3), activation='relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))
    self.model.add(Dropout(0.25))
    self.model.add(Flatten())
    self.model.add(Dense(128, activation='relu'))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(num_classes, activation='softmax'))
    
  def compile(self):
    self.model.compile(optimizer='adadelta', loss='categorical_crossentropy',
                       metrics=['accuracy'])
    
  def train(self, x, y):
    self.model.fit(x, y, batch_size=128, epochs=4)
    
  def evaluate(self, x, y):
    return self.model.evaluate(x, y)
    
  def select_best_predictions(self, x, y):
    predictions = self.model.predict(x, batch_size=128)    
    inputs, labels = list(), list()
    for i in range(len(predictions)): 
      if max(predictions[i]) > 0.95:
        inputs.append(x[i])
        labels.append(y[i])        
    return np.asarray(inputs), np.asarray(labels)

In [0]:
cnn = convnet()

In [0]:
cnn.compile()

In [0]:
cnn.train(train_seed, encode_y_train_seed)

Epoch 1/4
1000/1000 [==============================] - 7s 7ms/step - loss: 13.4275 - acc: 0.1390
Epoch 2/4
1000/1000 [==============================] - 5s 5ms/step - loss: 12.2404 - acc: 0.2220
Epoch 3/4
1000/1000 [==============================] - 3s 3ms/step - loss: 11.0453 - acc: 0.2950
Epoch 4/4
1000/1000 [==============================] - 4s 4ms/step - loss: 9.9485 - acc: 0.3600


In [0]:
new_x_train, new_y_train = cnn.select_best_predictions(train_unlabelled[:5000],
                            encode_y_train_unlabelled[:5000])

In [0]:
new_x_train.shape

(4814, 28, 28, 1)

In [0]:
new_y_train.shape

(4814, 10)

In [0]:
cnn.train(new_x_train, new_y_train)

Epoch 1/4
4814/4814 [==============================] - 27s 6ms/step - loss: 4.9312 - acc: 0.6132
Epoch 2/4
4814/4814 [==============================] - 27s 6ms/step - loss: 0.5330 - acc: 0.8498
Epoch 3/4
4814/4814 [==============================] - 27s 6ms/step - loss: 0.3338 - acc: 0.9055
Epoch 4/4
4814/4814 [==============================] - 27s 6ms/step - loss: 0.2225 - acc: 0.9344


In [0]:
print(cnn.model.evaluate(x_test, encode_y_test))

10000/10000 [==============================] - 16s 2ms/step
[0.1574420371491462, 0.9538]
